In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
CustDist_WHCentral = pd.read_csv("CustDist_WHCentral.csv", sep=",", decimal=".")
del CustDist_WHCentral["Distances between Customers and Warehouse"]
CustDist_WHCentral_list = CustDist_WHCentral.values.tolist()

CustDist_WHCorner = pd.read_csv("CustDist_WHCorner.csv", sep=",", decimal=".")
del CustDist_WHCorner["Distances between Customers and Warehouse"]
CustDist_WHCorner_list = CustDist_WHCorner.values.tolist()

CustOrd = pd.read_csv("CustOrd.csv", sep=",", decimal=".")
del CustOrd["Customer"]
CustOrd_list = CustOrd.values.tolist()

CustPos = pd.read_csv("CustXY_WHCentral.csv", sep=",", decimal=".")
del CustPos["Customer XY"]
CustPos_list = CustPos.values.tolist()

In [3]:
def evaluatecost(individual):
    total_distance = 0 
    load = 1000 
    return_to_wh = 0
    
    #falta ele ir ter de descarregar na última cena (corrigir)
    for i in range(len(individual)):
        #doesnt have enough load 
        if(load < CustOrd_list[individual[i]][0]):
            #last location -> central
            total_distance = total_distance + CustDist_WHCentral_list[0][individual[i-1]]
            load = 1000


            #central -> next location
            total_distance = total_distance + CustDist_WHCentral_list[0][individual[i]]
            load = load - CustOrd_list[individual[i]][0]
            
        
        #last client
        if(i >(len(individual) - 1)):
            #last location -> next location
            total_distance = total_distance + CustDist_WHCentral_list[individual[i]][individual[i-1]]
            load = load - CustOrd_list[individual[i]][0]
            #next location -> warehouse
            total_distance = total_distance + CustDist_WHCentral_list[individual[i]][0]
        
        #first client
        if(i == 0):
            #warehouse -> next location
            total_distance = total_distance + CustDist_WHCentral_list[0][individual[i]]
            load = load - CustOrd_list[individual[i]][0]
        
        #has enough load 
        else: 
            #last location -> next location
            total_distance = total_distance + CustDist_WHCentral_list[individual[i]][individual[i-1]]
            load = load - CustOrd_list[individual[i]][0]
            
    return total_distance,
def best_ind_plot(best_ind): # receives the best individual and adds the warehouse returns to the route to plot it

    total_distance = 0 
    load = 1000 

    for i in range(50):

        #last client
        if(i > 50):
            #last location -> next location
            total_distance = total_distance + CustDist_WHCentral_list[best_ind[i]][best_ind[i-1]]
            load = load - CustOrd_list[best_ind[i]][0]
            #next location -> warehouse
            total_distance = total_distance + CustDist_WHCentral_list[best_ind[i]][0]
            best_ind.insert(i, 0)
        
        #first client
        if(i == 0):
            #warehouse -> next location
            total_distance = total_distance + CustDist_WHCentral_list[0][best_ind[i]]
            load = load - CustOrd_list[best_ind[i]][0]

        #doesnt have enough load 
        if(load < CustOrd_list[best_ind[i]][0]):
            #last location -> central
            total_distance = total_distance + CustDist_WHCentral_list[0][best_ind[i-1]]
            load = 1000

            #append 0 in best_ind and shift right
            best_ind.insert(i, 0)
            

            #central -> next location
            total_distance = total_distance + CustDist_WHCentral_list[0][best_ind[i]]
            load = load - CustOrd_list[best_ind[i]][0]
            
        

        
        #has enough load 
        else: 
            #last location -> next location
            total_distance = total_distance + CustDist_WHCentral_list[best_ind[i]][best_ind[i-1]]
            load = load - CustOrd_list[best_ind[i]][0]
            
            
    return best_ind, total_distance
    
def plot_best_route_on_grid(best_ind_plot, dist): # receives the best individual and plots it on the grid
# plot the best route on the grid
    final_sol = best_ind_plot + best_ind_plot[0:1]    # add the wearhouse to the end of the list
    
    plt.figure(figsize=(10,10)) 

    plt.scatter([x[0] for x in CustPos_list], 
                [x[1] for x in CustPos_list], 
                s=50, c='g') # adds a green dot at costumers locations
    # add number to each costumer
    for i in range(50):
        plt.text(CustPos_list[i][0], CustPos_list[i][1], str(i), fontsize=15)
    
    plt.scatter(CustPos_list[0][0], CustPos_list[0][1], s=10, c='r')
    plt.scatter(CustPos_list[0][0], CustPos_list[0][1], s=10, c='w')
    plt.text(CustPos_list[0][0], CustPos_list[0][1], 'WH', fontsize=10)
    plt.grid()
    for p in range(len(final_sol) - 1):
        i = final_sol[p]
        j = final_sol[p+1]
        # if next customer is the warehouse change color to red
        if j == 0:
            color = 'red'
        else:
            color = 'black'       
        plt.arrow(CustPos_list[i][0], 
                CustPos_list[i][1],
                CustPos_list[j][0] - CustPos_list[i][0], 
                CustPos_list[j][1] - CustPos_list[i][1], 
                color=color)
        
    # add legend to the plot, and add a black arrow to the legend
    plt.legend(['Warehouse', 'Customers', 'Route'], loc='upper left')
    plt.arrow(CustPos_list[0][0], CustPos_list[0][1], 0.5, 0, color='black')

    
                

    plt.title('Best route found: ' + str(dist) + ' km')


               

    plt.show()

In [4]:
def heuristic():

    CustPos_list_XG50 = [] #list of customers with X > 50
    CustPos_list_XL50 = [] #list of customers with X < 50
    for customer in CustPos_list:
        
        if customer == [50,50]:
            pass
        elif customer[0] > 50:
            CustPos_list_XG50.append(customer)
        else:
            CustPos_list_XL50.append(customer)
    

    CustPos_list_XL50.sort(key=lambda x: x[1])

    CustPos_list_XG50.sort(key=lambda x: x[1], reverse=True)
    

    CustPos_list_XL50.extend(CustPos_list_XG50)

    
    
    heuristic_coords = CustPos_list_XL50
   
    heuristic_list = []
    for customer in heuristic_coords:
        heuristic_list.append(CustPos_list.index(customer))
    

    return heuristic_list
   
def add_heuristic_to_pop(population, heuristic_ind, flag): # receives the population and adds the heuristic solution to it
    if flag:    
        population.pop(len(population)-1)
        population.append(heuristic_ind)
        return population
    else: 
        return population


In [5]:
import random
import random
import numpy as np
from deap import base, creator, tools


def P1(Number_Customers, Population_Size, Number_of_Evaluations, seed):
    
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()

    heuristic_ind = heuristic()

    # permutation setup for individual,
    toolbox.register("indices", \
                     random.sample, \
                     range(Number_Customers), 
                     Number_Customers)

    

    toolbox.register("individual", \
                     tools.initIterate, \
                     creator.Individual, toolbox.indices)

    ## population setup,
    toolbox.register("population", \
                     tools.initRepeat, \
                     list, toolbox.individual)


    toolbox.register("mate", tools.cxOrdered)
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.01)
    toolbox.register("evaluate", evaluatecost)
    toolbox.register("select", tools.selTournament, tournsize=4)
    
    
    random.seed(seed)

    # create an initial population of 300 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=Population_Size)
    
    ## add heuristic solution to population


   
    print("---------POPULATION---------\n",pop,"\n-------------\n")
    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = .9, .1
    
    print("Start of evolution")
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    
    print("  Evaluated %i individuals" % len(pop))

    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0
    arr_mean = []
    arr_min = []
    # Begin the evolution
    while g < Number_of_Evaluations/ Population_Size:
        # A new generation
        g = g + 1
        #print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
    
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        #print("  Evaluated %i individuals" % len(invalid_ind))
        
        # The population is entirely replaced by the offspring
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        #print("  Min %s" % min(fits))
        #print("  Max %s" % max(fits))
        #print("  Avg %s" % mean)
        #print("  Std %s" % std)

        arr_mean.append(mean)
        arr_min.append(min(fits))
    
    #print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    
    x_coordinate = [i+1 for i in range(len(arr_mean))]
    plt.plot(x_coordinate, arr_mean, label = "Average")
    plt.plot(x_coordinate, arr_min, label = "Min")
    plt.title("Min and Average Fitness over generations")
    plt.legend()
    plt.grid()
    plt.figure()

    
    best_id_plot , dist= best_ind_plot(best_ind)
    plot_best_route_on_grid(best_id_plot, dist)

    
    print("HEURISTIC INDIVIDUAL: ", heuristic_ind)
    print(evaluatecost(heuristic_ind))
    heuristic_id_plot, dist = best_ind_plot(heuristic_ind)
    plot_best_route_on_grid(heuristic_id_plot, dist)

    return 

In [6]:
P1(51, 100, 10000, 42)

---------POPULATION---------
 [[40, 7, 1, 47, 17, 15, 14, 8, 6, 34, 5, 37, 27, 2, 48, 50, 13, 44, 32, 36, 46, 42, 22, 20, 28, 30, 41, 49, 33, 18, 43, 0, 35, 24, 10, 38, 39, 3, 12, 21, 31, 16, 29, 9, 26, 45, 4, 11, 19, 23, 25], [24, 5, 35, 18, 40, 39, 23, 36, 12, 4, 2, 14, 47, 49, 45, 6, 50, 17, 29, 44, 37, 11, 32, 48, 21, 8, 22, 26, 20, 46, 19, 30, 33, 7, 34, 43, 27, 41, 10, 28, 25, 3, 16, 0, 1, 38, 31, 9, 15, 42, 13], [36, 45, 20, 13, 41, 31, 25, 46, 29, 9, 16, 8, 15, 35, 34, 40, 27, 44, 23, 14, 4, 37, 38, 2, 24, 1, 3, 30, 48, 5, 47, 19, 50, 12, 17, 49, 39, 32, 28, 0, 10, 33, 42, 22, 6, 21, 18, 11, 43, 7, 26], [29, 0, 46, 48, 16, 32, 11, 45, 6, 40, 19, 43, 38, 12, 9, 23, 10, 33, 49, 20, 15, 39, 3, 44, 26, 25, 36, 7, 1, 35, 18, 2, 41, 30, 31, 4, 47, 34, 21, 8, 14, 17, 27, 42, 22, 24, 37, 13, 5, 28, 50], [33, 28, 7, 15, 14, 4, 21, 1, 37, 35, 46, 42, 40, 0, 45, 3, 38, 36, 2, 44, 32, 16, 48, 8, 31, 47, 6, 17, 50, 18, 26, 25, 49, 19, 13, 24, 43, 34, 10, 23, 5, 11, 9, 20, 27, 12, 39, 22, 41

IndexError: list assignment index out of range